In [11]:
import pandas as pd
import seaborn as sns

# Estudio de desempleo

## Paso 1: importado de datos CSV

In [18]:
#IMPORTAR CSV
df = pd.read_csv("Data/desempleo.csv")
print('Forma del Dataframe: ',df.shape)
print('Tamaño del Dataframe: ',df.size)

Forma del Dataframe:  (203520, 23)
Tamaño del Dataframe:  4680960


In [19]:
#ESCRIBIR CABECERA (5 ROWS)
df.head(5)

,OBS_STATUS_ID,OBS_STATUS_ID.1,OBS_STATUS_ID.2,OBS_STATUS_ID.3,OBS_STATUS_ID.4,OBS_STATUS_ID.5,OBS_STATUS_ID.6,OBS_STATUS_ID.7,OBS_STATUS_ID.8,OBS_STATUS_ID.9,...,OBS_STATUS_ID.13,OBS_STATUS_ID.14,OBS_STATUS_ID.15,OBS_STATUS_ID.16,OBS_STATUS_ID.17,OBS_STATUS_ID.18,OBS_STATUS_ID.19,OBS_STATUS_NAME,OBS_CONF_ID,OBS_CONF_NAME
0,ABW,Aruba,F,Female,Y15T24,15 to 24 years old,_T,Total,PT,Percentage,...,1960,NaN,WB_GS,Gender Statistics,0,Units,RATIO,Missing value,PU,Public
1,AFE,Africa Eastern and Southern,F,Female,Y15T24,15 to 24 years old,_T,Total,PT,Percentage,...,1960,NaN,WB_GS,Gender Statistics,0,Units,RATIO,Missing value,PU,Public
2,AFG,Afghanistan,F,Female,Y15T24,15 to 24 years old,_T,Total,PT,Percentage,...,1960,NaN,WB_GS,Gender Statistics,0,Units,RATIO,Missing value,PU,Public
3,AFW,Africa Western and Central,F,Female,Y15T24,15 to 24 years old,_T,Total,PT,Percentage,...,1960,NaN,WB_GS,Gender Statistics,0,Units,RATIO,Missing value,PU,Public
4,AGO,Angola,F,Female,Y15T24,15 to 24 years old,_T,Total,PT,Percentage,...,1960,NaN,WB_GS,Gender Statistics,0,Units,RATIO,Missing value,PU,Public


## Paso 2: Limpieza de datos

In [20]:
#EXTRAER EL AÑO 2019
df_2019 = df[df["OBS_STATUS_ID.13"] == 2019]
print('Forma del Dataframe: ',df_2019.shape)
print('Tamaño del Dataframe: ',df_2019.size)

Forma del Dataframe:  (3180, 23)
Tamaño del Dataframe:  73140


In [21]:
# Lista simple de países europeos
europa = [
    "Spain","France","Germany","Italy","Portugal","Belgium","Netherlands","Sweden",
    "Norway","Denmark","Finland","Poland","Austria","Greece","Ireland","Czechia",
    "Slovakia","Slovenia","Hungary","Romania","Bulgaria","Croatia","Estonia",
    "Latvia","Lithuania","Luxembourg","Malta","Iceland","Switzerland","Serbia",
    "Ukraine","Russia","Turkey","Belarus","North Macedonia"
]

# Filtrar países europeos presentes en el dataset
df_europa = df_2019[df_2019["OBS_STATUS_ID.1"].isin(europa)]

In [22]:
df_europa = df_europa.drop(columns=[
    "OBS_STATUS_ID.2",
    "OBS_STATUS_ID.4",
    "OBS_STATUS_ID.6",
    "OBS_STATUS_ID.7",
    "OBS_STATUS_ID.8",
    "OBS_STATUS_ID.9",
    "OBS_STATUS_ID.10",
    "OBS_STATUS_ID.11",
    "OBS_STATUS_ID.12",
    "OBS_STATUS_ID.13",
    "OBS_STATUS_ID.15",
    "OBS_STATUS_ID.16",
    "OBS_STATUS_ID.17",
    "OBS_STATUS_ID.18",
    "OBS_STATUS_ID.19",
    "OBS_STATUS_NAME",
    "OBS_CONF_ID",
    "OBS_CONF_NAME"
])


In [23]:
df_europa.head(10)


,OBS_STATUS_ID,OBS_STATUS_ID.1,OBS_STATUS_ID.3,OBS_STATUS_ID.5,OBS_STATUS_ID.14
187634,AUT,Austria,Female,15 to 24 years old,7944.0
187637,BEL,Belgium,Female,15 to 24 years old,12275.0
187641,BGR,Bulgaria,Female,15 to 24 years old,8322.0
187645,BLR,Belarus,Female,15 to 24 years old,7344.0
187657,CHE,Switzerland,Female,15 to 24 years old,7166.0
187674,CZE,Czechia,Female,15 to 24 years old,5999.0
187675,DEU,Germany,Female,15 to 24 years old,4756.0
187678,DNK,Denmark,Female,15 to 24 years old,9733.0
187690,ESP,Spain,Female,15 to 24 years old,34474.0
187691,EST,Estonia,Female,15 to 24 years old,12098.0


## Paso 3: análisis

### Muestra de promedio de consumo en gramos por país

In [ ]:
#UNA VEZ LIMPIO, PRETENDO MOSTRAR UN RESUMEN DE MEDIA POR PAIS

df_resumen=df_limpio.groupby("country")["value"].mean().reset_index()
df_resumen.set_index("country", inplace=True)
df_resumen.sort_values(by="value", ascending=False, inplace=True)   
print('Resumen de los primeros cinco paises con mayor consumo diario de frutas y verduras:')
print(df_resumen.head())
print("_"*50)
print('Resumen de los últimos cinco paises con menor consumo diario de frutas y verduras:')
print(df_resumen.tail())

## Paso 4: comunicación de datos y visualización

### Mostramos gráficos de consumo por país

In [ ]:
#MUESTRO UN GRÁFICO PARA VER MEJOR LA DIFERENCIA ENTRE PAÍSES

paises = df_resumen.index
consumo = df_resumen["value"]

plt.figure(figsize=(12, 6))
plt.bar(paises, consumo, color='skyblue')
plt.xlabel('Países')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.title('Consumo Promedio de Frutas y Verduras por País en Europa')
plt.tight_layout()
plt.show()

In [ ]:
#OTRO GRÁFICO DE BARRAS CON LÍMITES MÁS JUSTOS
plt.figure(figsize=(12, 6))
plt.bar(paises, consumo)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(np.min(consumo), np.max(consumo)+0.2)  # Ajusta los límites del eje y para enfocarte en el rango relevante
plt.xlabel('Países')    
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.title('Consumo Promedio de Frutas y Verduras por País en Europa (Enfoque)')
plt.tight_layout()      
plt.show()

### Analizamos el sexo en relación al consumo y mostramos gráfico

In [ ]:
#AHORA QUIERO VER LA DIFERENCIA ENTRE SEXOS EN CADA PAÍS
#AGRUPO POR SEXOS Y DESPUÉS HAGO UN PIVOT PARA TENERLOS EN COLUMNAS
df_sexo = df_limpio.groupby(["country","sex"])["value"].mean().reset_index()
df_sexo_pivot = df_sexo.pivot(index="country", columns="sex", values="value")
df_sexo_pivot["diferencia"] = df_sexo_pivot.get("F", pd.Series(dtype=float)) - df_sexo_pivot.get("M", pd.Series(dtype=float))
df_sexo_pivot.sort_values(by="diferencia", ascending=False, inplace=True)

#EXTRAIGO ARRAYS PARA EL GRÁFICO
f = df_sexo_pivot["F"]
m = df_sexo_pivot["M"]

paises = df_sexo_pivot.index
x = np.arange(len(paises))

plt.figure(figsize=(12, 6))
plt.plot(x, f, label='Mujeres (F)', marker='o', color='pink')
plt.plot(x, m, label='Hombres (M)', marker='o')
plt.xticks(x, paises)
plt.title('Consumo Promedio de Frutas y Verduras por País y Sexo en Europa')
plt.xlabel('Países')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.legend()

plt.tight_layout()
plt.show()


### Análisis de consumo por rangos de edades

In [ ]:
#VAMOS A HACER UN ANÁLISIS POR EDADES TAMBIÉN
df_edad = df_limpio.groupby(["age"])["value"].mean()
print(df_edad.head())


In [ ]:
#GRAFICO DE BARRAS PARA ANALIZAR POR RANGOS DE EDADES
rangos_edad=df_edad.index
consumo_edad=df_edad.values

# Incrementos de 0.05 para máxima granularidad
yticks = np.arange(min(consumo_edad), max(consumo_edad) + 0.05, 0.05)

plt.figure(figsize=(10, 5))
#limites más justos para ver mejor la diferencia
plt.ylim(33.3, 33.65)
plt.bar(rangos_edad, consumo_edad, color='orange')
plt.xlabel('Rangos de Edad')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)') 
plt.title('Consumo Promedio de Frutas y Verduras por Rango de Edad en Europa')
plt.yticks(yticks)
plt.tight_layout()
plt.show()

## Paso extra: preparación de datos para uso de compañeros

### Aquí escribo un resumen de los dataframes y series utilizados en limpio para facilitar la extracción de los mismos por parte de mis compañeros

In [ ]:
#DATAFRAMES EXTRAIDOS
df_final=df_limpio.copy()
df_paises = df_resumen.copy()
df_sexos = df_sexo_pivot.copy()
df_edades = df_edad.copy()
#_______________________________________________________________________________________
print('RESUMEN DE DATAFRAMES EXTRAIDOS:')
print("-"*50)

print("DATAFRAME LIMPIO:")
print(df_final.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR PAÍS:")
print(df_paises.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR SEXO Y PAÍS:")
print(df_sexos.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR RANGO DE EDAD:")
print(df_edades.head())
print("_"*50)

In [ ]:
#SERIES EXTRAIDAS
serie_paises = df_resumen["value"].copy() 
serie_hombres = df_sexo_pivot["M"].copy()
serie_mujeres = df_sexo_pivot["F"].copy() 
serie_consumo_edad = df_edad.values.copy()